In [1]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
import sklearn.metrics as skm
import torch
import cv2
import yaml
import albumentations as A

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False

fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
DIR_DATA = '/kaggle/input/ghana-crop-disease'
df = pd.read_csv(f'{DIR_DATA}/Train.csv')
CLASSES = sorted(df['class'].unique())
CLASSES = {c:i for i,c in enumerate(CLASSES)}
print(CLASSES)
df['path'] = f'{DIR_DATA}/images/'+df.Image_ID
df['category_id'] = df['class'].map(CLASSES).fillna(-1)
df['crop'] = df['class'].str.split('_').str[0]
d = df.groupby('Image_ID')['crop'].nunique()
df = pd.merge(df,pd.DataFrame({'Image_ID':d.index,'n_crop':d.values}),on='Image_ID',how='left')
df

{'Corn_Cercospora_Leaf_Spot': 0, 'Corn_Common_Rust': 1, 'Corn_Healthy': 2, 'Corn_Northern_Leaf_Blight': 3, 'Corn_Streak': 4, 'Pepper_Bacterial_Spot': 5, 'Pepper_Cercospora': 6, 'Pepper_Early_Blight': 7, 'Pepper_Fusarium': 8, 'Pepper_Healthy': 9, 'Pepper_Late_Blight': 10, 'Pepper_Leaf_Blight': 11, 'Pepper_Leaf_Curl': 12, 'Pepper_Leaf_Mosaic': 13, 'Pepper_Septoria': 14, 'Tomato_Bacterial_Spot': 15, 'Tomato_Early_Blight': 16, 'Tomato_Fusarium': 17, 'Tomato_Healthy': 18, 'Tomato_Late_Blight': 19, 'Tomato_Leaf_Curl': 20, 'Tomato_Mosaic': 21, 'Tomato_Septoria': 22}


,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,/kaggle/input/ghana-crop-disease/images/id_11e...,8,Pepper,1
...,...,...,...,...,...,...,...,...,...,...,...
41024,id_38x7ln.jpg,1.0,Corn_Streak,608.472222,817.741770,736.352881,997.047325,/kaggle/input/ghana-crop-disease/images/id_38x...,4,Corn,1
41025,id_7uqmqp.jpg,1.0,Corn_Healthy,1483.095425,818.249483,1786.752393,1152.860096,/kaggle/input/ghana-crop-disease/images/id_7uq...,2,Corn,1
41026,id_bnqorw.jpg,1.0,Corn_Cercospora_Leaf_Spot,943.260816,1015.079256,1007.928503,1133.177119,/kaggle/input/ghana-crop-disease/images/id_bnq...,0,Corn,1
41027,id_m2kklk.jpg,1.0,Tomato_Septoria,1855.337090,1748.733977,2046.688525,1868.580553,/kaggle/input/ghana-crop-disease/images/id_m2k...,22,Tomato,1


In [3]:
df.crop.value_counts()

crop
Tomato    17156
Corn      13128
Pepper    10745
Name: count, dtype: int64

In [4]:
df = df[df['crop'] != 'Corn']
df = df.reset_index(drop=True)


In [5]:
df

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,/kaggle/input/ghana-crop-disease/images/id_11e...,8,Pepper,1
...,...,...,...,...,...,...,...,...,...,...,...
27896,id_bgm8xw.jpg,1.0,Pepper_Leaf_Mosaic,1734.895719,3265.010031,1981.568332,3497.658179,/kaggle/input/ghana-crop-disease/images/id_bgm...,13,Pepper,1
27897,id_c7lws4.jpg,1.0,Pepper_Bacterial_Spot,1913.940651,961.942983,1991.450533,1056.455069,/kaggle/input/ghana-crop-disease/images/id_c7l...,5,Pepper,1
27898,id_tw4lex.jpg,1.0,Pepper_Bacterial_Spot,1110.008844,2232.019671,1193.680719,2291.961126,/kaggle/input/ghana-crop-disease/images/id_tw4...,5,Pepper,1
27899,id_m2kklk.jpg,1.0,Tomato_Septoria,1855.337090,1748.733977,2046.688525,1868.580553,/kaggle/input/ghana-crop-disease/images/id_m2k...,22,Tomato,1


In [6]:
df.describe()

,confidence,ymin,xmin,ymax,xmax,category_id,n_crop
count,27901.0,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000
mean,1.0,1187.015155,1599.322623,1376.917084,1804.155914,16.008709,1.214114
std,0.0,736.642289,937.692304,771.914279,981.480618,5.131521,0.410214
min,1.0,0.000000,0.000000,30.998991,17.225673,5.000000,1.000000
25%,1.0,576.523046,869.634298,714.815789,1002.458592,13.000000,1.000000
50%,1.0,1114.584309,1489.008077,1316.572254,1710.720121,17.000000,1.000000
75%,1.0,1732.780123,2260.502696,1955.386132,2510.273103,20.000000,1.000000
max,1.0,4295.942029,5908.600181,4459.818841,6567.427798,22.000000,2.000000


In [7]:
def split_data(df):

    gkf  = StratifiedGroupKFold(n_splits=N_SPLITS,shuffle=True,random_state=RANDOM_STATE)

    df['fold'] = -1

    for fold_id, (train_index, test_index) in enumerate(gkf.split(df,y=df['class'],groups=df.Image_ID)):

        df.loc[test_index,'fold'] = fold_id

    return df


df = split_data(df)
df

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop,fold
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,/kaggle/input/ghana-crop-disease/images/id_11e...,8,Pepper,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27896,id_bgm8xw.jpg,1.0,Pepper_Leaf_Mosaic,1734.895719,3265.010031,1981.568332,3497.658179,/kaggle/input/ghana-crop-disease/images/id_bgm...,13,Pepper,1,1
27897,id_c7lws4.jpg,1.0,Pepper_Bacterial_Spot,1913.940651,961.942983,1991.450533,1056.455069,/kaggle/input/ghana-crop-disease/images/id_c7l...,5,Pepper,1,1
27898,id_tw4lex.jpg,1.0,Pepper_Bacterial_Spot,1110.008844,2232.019671,1193.680719,2291.961126,/kaggle/input/ghana-crop-disease/images/id_tw4...,5,Pepper,1,0
27899,id_m2kklk.jpg,1.0,Tomato_Septoria,1855.337090,1748.733977,2046.688525,1868.580553,/kaggle/input/ghana-crop-disease/images/id_m2k...,22,Tomato,1,4


In [8]:
df['class'].value_counts()

class
Tomato_Septoria          6483
Tomato_Late_Blight       3918
Tomato_Healthy           2872
Pepper_Septoria          2295
Pepper_Leaf_Mosaic       2113
Tomato_Early_Blight      1946
Pepper_Bacterial_Spot    1943
Pepper_Leaf_Curl         1530
Tomato_Fusarium           882
Pepper_Healthy            736
Pepper_Late_Blight        575
Pepper_Leaf_Blight        508
Tomato_Bacterial_Spot     484
Pepper_Fusarium           472
Pepper_Cercospora         472
Tomato_Leaf_Curl          437
Tomato_Mosaic             134
Pepper_Early_Blight       101
Name: count, dtype: int64

In [9]:
cls = ['Pepper_Early_Blight', 'Pepper_Late_Blight', 'Pepper_Septoria']
df_pepper = df[df['class'].isin(cls)].copy()
df = df[~((df.n_crop>1)&(df.crop=='Tomato'))].copy()

CLASSES = sorted(df['class'].unique())
CLASSES = {c:i for i,c in enumerate(CLASSES)}
print(len(CLASSES),CLASSES)



18 {'Pepper_Bacterial_Spot': 0, 'Pepper_Cercospora': 1, 'Pepper_Early_Blight': 2, 'Pepper_Fusarium': 3, 'Pepper_Healthy': 4, 'Pepper_Late_Blight': 5, 'Pepper_Leaf_Blight': 6, 'Pepper_Leaf_Curl': 7, 'Pepper_Leaf_Mosaic': 8, 'Pepper_Septoria': 9, 'Tomato_Bacterial_Spot': 10, 'Tomato_Early_Blight': 11, 'Tomato_Fusarium': 12, 'Tomato_Healthy': 13, 'Tomato_Late_Blight': 14, 'Tomato_Leaf_Curl': 15, 'Tomato_Mosaic': 16, 'Tomato_Septoria': 17}


In [10]:
df['class'].value_counts()

class
Tomato_Septoria          4166
Tomato_Late_Blight       3343
Tomato_Healthy           2872
Pepper_Septoria          2295
Pepper_Leaf_Mosaic       2113
Pepper_Bacterial_Spot    1943
Tomato_Early_Blight      1845
Pepper_Leaf_Curl         1530
Tomato_Fusarium           877
Pepper_Healthy            736
Pepper_Late_Blight        575
Pepper_Leaf_Blight        508
Tomato_Bacterial_Spot     484
Pepper_Fusarium           472
Pepper_Cercospora         472
Tomato_Leaf_Curl          437
Tomato_Mosaic             134
Pepper_Early_Blight       101
Name: count, dtype: int64

In [11]:
for fold in range(N_SPLITS):
  d=df[df.fold==fold]['class'].value_counts().to_dict()
  print(fold,d)

0 {'Tomato_Septoria': 875, 'Tomato_Late_Blight': 708, 'Tomato_Healthy': 643, 'Pepper_Bacterial_Spot': 457, 'Tomato_Early_Blight': 409, 'Pepper_Leaf_Mosaic': 391, 'Pepper_Septoria': 332, 'Pepper_Leaf_Curl': 328, 'Pepper_Healthy': 170, 'Tomato_Bacterial_Spot': 138, 'Tomato_Fusarium': 135, 'Pepper_Fusarium': 114, 'Pepper_Late_Blight': 104, 'Tomato_Leaf_Curl': 79, 'Pepper_Leaf_Blight': 64, 'Pepper_Cercospora': 52, 'Pepper_Early_Blight': 28, 'Tomato_Mosaic': 20}
1 {'Tomato_Septoria': 1036, 'Tomato_Late_Blight': 574, 'Pepper_Septoria': 528, 'Tomato_Healthy': 505, 'Pepper_Leaf_Mosaic': 463, 'Pepper_Bacterial_Spot': 423, 'Tomato_Early_Blight': 349, 'Pepper_Leaf_Curl': 288, 'Tomato_Fusarium': 225, 'Pepper_Healthy': 123, 'Pepper_Cercospora': 116, 'Pepper_Fusarium': 115, 'Pepper_Leaf_Blight': 115, 'Pepper_Late_Blight': 104, 'Tomato_Leaf_Curl': 72, 'Tomato_Bacterial_Spot': 50, 'Tomato_Mosaic': 24, 'Pepper_Early_Blight': 16}
2 {'Tomato_Late_Blight': 821, 'Tomato_Septoria': 696, 'Tomato_Early_Blight

# Create YOLO format dataset

In [14]:
DATASET_DIRECTORY = '/kaggle/input/ghana-crop-disease'
os.makedirs(DATASET_DIRECTORY,exist_ok=True)

In [15]:
df_val = df[df.fold==FOLD]
df_val

,Image_ID,confidence,class,ymin,xmin,ymax,xmax,path,category_id,crop,n_crop,fold
0,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,194.649671,328.803454,208.107730,341.967928,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
1,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,149.632401,256.768914,162.910362,266.195724,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
2,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,234.046875,327.138158,252.712993,338.876645,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
3,id_11543h.jpg,1.0,Pepper_Bacterial_Spot,221.277138,340.411184,238.593750,354.651316,/kaggle/input/ghana-crop-disease/images/id_115...,5,Pepper,1,0
4,id_11ee1c.jpg,1.0,Pepper_Fusarium,2000.563598,989.588908,2184.252196,1401.748952,/kaggle/input/ghana-crop-disease/images/id_11e...,8,Pepper,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27870,id_8sfigs.jpg,1.0,Tomato_Early_Blight,489.121780,964.244731,586.247073,1015.386417,/kaggle/input/ghana-crop-disease/images/id_8sf...,16,Tomato,1,0
27880,id_jn2hs3.jpg,1.0,Tomato_Bacterial_Spot,465.722336,2029.260975,582.049180,2144.862379,/kaggle/input/ghana-crop-disease/images/id_jn2...,15,Tomato,1,0
27887,id_rdbt4i.jpg,1.0,Pepper_Leaf_Blight,2517.801832,3791.284642,2632.473290,3859.623268,/kaggle/input/ghana-crop-disease/images/id_rdb...,11,Pepper,1,0
27895,id_dhb15p.jpg,1.0,Tomato_Fusarium,1997.330411,2752.593539,2175.784381,3330.610255,/kaggle/input/ghana-crop-disease/images/id_dhb...,17,Tomato,1,0


In [19]:
ID2CLASS = {v:k for k,v in CLASSES.items()}

conf = {
    'path': './',
    'train': 'train/images',
    'val': 'valid/images',
    'nc': len(CLASSES),
    'names': ID2CLASS
}
with open('/kaggle/working/config.yaml','w') as f:
    yaml.dump(conf,f,default_flow_style=False)

display(conf)

{'path': './',
 'train': 'train/images',
 'val': 'valid/images',
 'nc': 18,
 'names': {0: 'Pepper_Bacterial_Spot',
  1: 'Pepper_Cercospora',
  2: 'Pepper_Early_Blight',
  3: 'Pepper_Fusarium',
  4: 'Pepper_Healthy',
  5: 'Pepper_Late_Blight',
  6: 'Pepper_Leaf_Blight',
  7: 'Pepper_Leaf_Curl',
  8: 'Pepper_Leaf_Mosaic',
  9: 'Pepper_Septoria',
  10: 'Tomato_Bacterial_Spot',
  11: 'Tomato_Early_Blight',
  12: 'Tomato_Fusarium',
  13: 'Tomato_Healthy',
  14: 'Tomato_Late_Blight',
  15: 'Tomato_Leaf_Curl',
  16: 'Tomato_Mosaic',
  17: 'Tomato_Septoria'}}

In [21]:
def normalize_labels(x,w = None, h=None):
    x = x.astype(np.float32)
    y = np.zeros_like(x).astype(np.float32)
    y[0] = (x[0] + x[2]) / 2 / w  # x center
    y[1] = (x[1] + x[3]) / 2 / h  # y center
    y[2] = (x[2] - x[0]) / w  # box width
    y[3] = (x[3] - x[1]) / h  # box height
    return y


In [25]:
df['split'] = 'train'
df.loc[df.fold==FOLD,'split'] = 'val'
df.split.value_counts()

split
train    19856
val       5047
Name: count, dtype: int64

In [29]:
def prep_ds(meta, split):
    out_dir = f'/kaggle/working/{split}'
    shutil.rmtree(out_dir,ignore_errors=True)
    img_dir = f'{out_dir}/images'
    label_dir = f'{out_dir}/labels'
    os.makedirs(img_dir)
    os.makedirs(label_dir)

    # training on complete dataset to maintan consistency with the submitted model
    # comment for realistic validation
    if split=='train':
        df = meta.copy()
    else:
        df = meta[meta.split==split].copy()

    df.to_csv(f'{out_dir}/submission_format.csv',index=False)
    print(f'writing {split} data to {out_dir} images: {df.Image_ID.nunique()} annotations: {len(df)}')
    for image_id,d in tqdm(df.groupby('Image_ID')):
      path = d.path.values[0]
      d = d.dropna(subset=['xmin','ymin','xmax','ymax','category_id'])
      #skip bg only images
      if len(d)==0:
        print(f'skipping {image_id}')

      shutil.copy2(path, f"{img_dir}/{path.split('/')[-1]}")
      im = cv2.imread(path)
      height,width,_ = im.shape
      image_id = image_id.split('.')[0]
      for _, row in d.iterrows():
          if row.category_id==-1:
            continue
          x0, y0, x1, y1 = row.xmin,row.ymin,row.xmax,row.ymax
          w = x1 - x0
          h = y1 - y0
          x = np.array([x0,y0,x1,y1]) #['xmin', 'ymin', 'xmax', 'ymax']
          if w<1 or h<1:
            print(row)
            continue

          xnorm = normalize_labels(x,w=width,h=height)
          assert (xnorm>0).all()
          xnorm = xnorm.astype(str)
          # print(x,xnorm,height,width)
          with open(f"{label_dir}/{image_id}.txt",'a') as f:
              f.write(f"{int(row.category_id)}\t"+'\t'.join(xnorm)+'\n')

def gen_data(fold,meta):
    meta['split'] = 'train'
    meta.loc[meta.fold==fold,'split'] = 'valid'
    display(meta.split.value_counts())
    for split in ['train','valid']:
        prep_ds(meta,split)


gen_data(FOLD,df)

split
train    19856
valid     5047
Name: count, dtype: int64

writing train data to /kaggle/working/train images: 3493 annotations: 24903


100%|██████████| 3493/3493 [06:31<00:00,  8.91it/s]


writing valid data to /kaggle/working/valid images: 698 annotations: 5047


100%|██████████| 698/698 [00:57<00:00, 12.18it/s]


In [ ]:
# !zip -rqq {DATASET_DIRECTORY}.zip {DATASET_DIRECTORY}

In [33]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 15.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
from ultralytics.data.build import YOLODataset
import ultralytics.data.build as build

class YOLOWeightedDataset(YOLODataset):
    def __init__(self, *args, mode="train", **kwargs):
        """
        Initialize the WeightedDataset.

        Args:
            class_weights (list or numpy array): A list or array of weights corresponding to each class.
        """

        super(YOLOWeightedDataset, self).__init__(*args, **kwargs)

        self.train_mode = "train" in self.prefix

        # You can also specify weights manually instead
        self.count_instances()
        class_weights = np.sum(self.counts) / self.counts

        # Aggregation function
        self.agg_func = np.mean

        self.class_weights = np.array(class_weights)
        self.weights = self.calculate_weights()
        self.probabilities = self.calculate_probabilities()

    def count_instances(self):
        """
        Count the number of instances per class

        Returns:
            dict: A dict containing the counts for each class.
        """
        self.counts = [0 for i in range(len(self.data["names"]))]
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)
            for id in cls:
                self.counts[id] += 1

        self.counts = np.array(self.counts)
        self.counts = np.where(self.counts == 0, 1, self.counts)

    def calculate_weights(self):
        """
        Calculate the aggregated weight for each label based on class weights.

        Returns:
            list: A list of aggregated weights corresponding to each label.
        """
        weights = []
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)

            # Give a default weight to background class
            if cls.size == 0:
              weights.append(1)
              continue

            # Take mean of weights
            # You can change this weight aggregation function to aggregate weights differently
            weight = self.agg_func(self.class_weights[cls])
            weights.append(weight)
        return weights

    def calculate_probabilities(self):
        """
        Calculate and store the sampling probabilities based on the weights.

        Returns:
            list: A list of sampling probabilities corresponding to each label.
        """
        total_weight = sum(self.weights)
        probabilities = [w / total_weight for w in self.weights]
        return probabilities

    def __getitem__(self, index):
        """
        Return transformed label information based on the sampled index.
        """
        # Don't use for validation
        if not self.train_mode:
            return self.transforms(self.get_image_and_label(index))
        else:
            index = np.random.choice(len(self.labels), p=self.probabilities)
            return self.transforms(self.get_image_and_label(index))

build.YOLODataset = YOLOWeightedDataset

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [39]:
from ultralytics import YOLO

IMGSZ = 1024
run_dir = f'train_{FOLD}'
print(run_dir)
!rm -rf runs/detect/{run_dir}

MODEL_NAME = 'yolov5s'
model = YOLO(f'{MODEL_NAME}.pt')
results = model.train(data='/kaggle/input/configg/config.yaml',
                      epochs=50,
                      imgsz=IMGSZ,
                      name=run_dir,
                      device=0,
                      batch=20,
                      optimizer='AdamW',
                      lr0=3e-4,
                      momentum=0.9,  
                      weight_decay=1e-2,
                      close_mosaic=30,
                      seed=RANDOM_STATE)

train_0
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 158MB/s]


Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/kaggle/input/configg/config.yaml, epochs=50, time=None, patience=100, batch=20, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train_0, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=41, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=30, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning /kaggle/working/train/labels.cache... 3493 images, 0 backgrounds, 889 corrupt: 100%|██████████| 3493/3493 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/working/train/images/id_16o914.jpg: ignoring corrupt image/label: Label class 18 exceeds dataset class count 18. Possible class labels are 0-17
train: WARNING ⚠️ /kaggle/working/train/images/id_17lyu5.jpg: ignoring corrupt image/label: Label class 22 exceeds dataset class count 18. Possible class labels are 0-17
train: WARNING ⚠️ /kaggle/working/train/images/id_1csdxd.jpg: ignoring corrupt image/label: Label class 18 exceeds dataset class count 18. Possible class labels are 0-17
train: WARNING ⚠️ /kaggle/working/train/images/id_1dkvrb.jpg: ignoring corrupt image/label: Label class 20 exceeds dataset class count 18. Possible class labels are 0-17
train: WARNING ⚠️ /kaggle/working/train/images/id_1e8ow1.jpg: ignoring corrupt image/label: Label class 22 exceeds dataset class count 18. Possible class labels are 0-17
train: WARNING ⚠️ /kaggle/working/train/images/id_1gfauc.jpg: ignoring corrupt image/label: Label class 22 exceeds dataset class count 18. Possible cl

...


50 epochs completed in 2.043 hours.
Optimizer stripped from runs/detect/train_0/weights/last.pt, 18.6MB
Optimizer stripped from runs/detect/train_0/weights/best.pt, 18.6MB

Validating runs/detect/train_0/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv5s summary (fused): 193 layers, 9,118,502 parameters, 0 gradients, 23.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:19<00:00,  1.38s/it]


                   all        522       2473      0.752      0.751      0.777      0.482
    Pepper_Late_Blight        122        457      0.695        0.3      0.463      0.152
    Pepper_Leaf_Blight         20         52      0.613      0.769      0.685      0.275
      Pepper_Leaf_Curl          3         28      0.938          1      0.995      0.847
    Pepper_Leaf_Mosaic         54        114      0.893      0.991      0.992      0.724
       Pepper_Septoria         63        170      0.775      0.965      0.958      0.707
 Tomato_Bacterial_Spot          7        104      0.768      0.827      0.873      0.563
   Tomato_Early_Blight         26         64      0.724      0.859      0.863      0.489
       Tomato_Fusarium        116        328      0.699      0.683      0.737      0.395
        Tomato_Healthy        117        391      0.648     0.0614      0.173      0.065
    Tomato_Late_Blight         22        332      0.669       0.53       0.61      0.269
      Tomato_Leaf_Cur

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.3ms preprocess, 7.6ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/train_0


Accuracy : 75%

# Saving Model

In [40]:
# Define the path where the model is saved
trained_model_path = f'runs/detect/{run_dir}/weights/best.pt'

# Copy it to a desired location, e.g., "/kaggle/working/"
shutil.copy(trained_model_path, "/kaggle/working/best.pt")


'/kaggle/working/best.pt'